In [17]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
import pickle
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

from util import CustomLoss, CustomFeaturesDataset

import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_dir = Path("../../features_model/{date:%d-%m_%H-%M}".format(date=datetime.datetime.now()))
save_dir.mkdir(parents=True, exist_ok=True)

In [18]:
#### Load and process dataset ####
with open("../../data/dataset10000.pkl", "rb") as f:
    data = pickle.load(f)

xs = data[:,-3]
ys = data[:,-2]
print("[", np.min(xs), np.max(xs), "]")
print("[", np.min(ys), np.max(ys), "]")

# Normalize distance data
data[:,-3] = (1/40)*(xs+20)
data[:,-2] = (1/48)*(ys+24)
print("[", np.min(data[:,-3]), np.max(data[:,-3]), "]")
print("[", np.min(data[:,-2]), np.max(data[:,-2]), "]")



(20000, 1445)
[ -19.973097473190375 19.96787178502315 ]
[ -20.129999801800473 23.61255961029876 ]
[ 0.0006725631702406255 0.9991967946255788 ]
[ 0.08062500412915681 0.9919283252145574 ]


In [19]:
print(data.shape)
(n_samples, sample_dims) = data.shape
batch_size = 50

# Convert dataset and split it into training, validation and testing
dataset = CustomFeaturesDataset(data, 3)
train, valid, test = random_split(dataset,[3/4, 1/8, 1/8])

# Save datasets
with open(save_dir / "train_dataset.pkl", "wb") as f:
    pickle.dump(train, f)
with open(save_dir / "valid_dataset.pkl", "wb") as f:
    pickle.dump(valid, f)
with open(save_dir / "test_dataset.pkl", "wb") as f:
    pickle.dump(test, f)

# Create dataloaders
trainloader = DataLoader(train, batch_size=batch_size)
validloader = DataLoader(valid, batch_size=batch_size)
testloader = DataLoader(test, batch_size=int(n_samples/8))

In [20]:
#### Defining parameters and model ####
num_epochs = 1000

model = nn.Sequential(
    nn.Linear(721*2, 32),
    nn.ReLU(),
    nn.Linear(32, 3),
    nn.Sigmoid()
)
model.to(device)

loss_function = CustomLoss()
loss_function.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

In [21]:
#### Training ####
train_loss = []
valid_loss = []
valid_loss_lowpass = []
window = 50

prev_saved= False
for epoch in trange(num_epochs):
   
    #Train set
    epoch_loss = 0
    for batch_in, batch_feats in trainloader:
        # Forward pass 
        outputs = model(batch_in)
        loss = loss_function(outputs, batch_feats)
        epoch_loss += loss

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss.append(epoch_loss)

    # Validation set
    epoch_loss = 0
    for batch_in, batch_feats in validloader:
        outputs = model(batch_in)
        loss = loss_function(outputs, batch_feats)
        epoch_loss += float(loss)
    valid_loss.append(epoch_loss)
    valid_loss_lowpass.append(np.mean(valid_loss[epoch-window:epoch]))
    
    # If the validation error is going up, save model
    if epoch > window+10 and valid_loss_lowpass[epoch-1]>max(valid_loss_lowpass[epoch-10:epoch-2]) and not prev_saved:
        torch.save(model, save_dir / f'model_{epoch}.pth')
        prev_saved = True
    elif epoch > window+10 and valid_loss_lowpass[epoch-1]<=max(valid_loss_lowpass[epoch-10:epoch-2]):
        prev_saved = False
 


  0%|          | 0/1000 [00:00<?, ?it/s]

c:\Users\Louise\Documents\EPFL\UPRAMDYA_intern\Flygym\flyvenv\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Louise\Documents\EPFL\UPRAMDYA_intern\Flygym\flyvenv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1000/1000 [40:38<00:00,  2.44s/it]


In [22]:
#### Saving and plotting results ####
torch.save(model, save_dir / f'model_{epoch}.pth')

with open(save_dir / "train_loss.pkl", "wb") as f:
    pickle.dump(train_loss, f)
with open(save_dir / "valid_loss.pkl", "wb") as f:
    pickle.dump(valid_loss, f)


In [23]:
LOAD = False
if LOAD :
    load_dir = save_dir #Path("../../features_model/11-08_10-02")

    with open(load_dir / "train_loss.pkl", "rb") as f:
        train_loss = pickle.load(f)
    with open(load_dir / "valid_loss.pkl", "rb") as f:
        valid_loss = pickle.load(f)

n_train_samples = 3*n_samples/4
n_valid_samples = n_samples/8

train_loss = torch.tensor(train_loss).detach().numpy()
valid_loss = torch.tensor(valid_loss).detach().numpy()

plt.plot(range(num_epochs), train_loss/n_train_samples)
plt.plot(range(num_epochs), valid_loss/n_valid_samples)
plt.show()

window = 50
avg_train = [np.mean(train_loss[i:i+window])/n_train_samples for i in range(len(train_loss)-window)]
avg_valid = [np.mean(valid_loss[i:i+window])/n_valid_samples for i in range(len(valid_loss)-window)]

print(np.argmin(avg_train)+window)
print(np.argmin(avg_valid)+window)


plt.plot(range(window, num_epochs), avg_train)
plt.plot(range(window, num_epochs), avg_valid)
plt.show()

NameError: name 'save_path' is not defined

In [16]:
n_test_samples = n_samples/8

test_loss = []
for num in [698]:
    with open(load_dir / f"model_{num}.pth","rb") as f:
        model = torch.load(f)
    model.eval()

    tot_loss = 0
    for test_in, test_feats in testloader:
        outputs = model(test_in)
        loss = loss_function(outputs, test_feats)

    test_feats[:,-3] = 40*test_feats[:,-3]-20
    test_feats[:,-2] = 48*test_feats[:,-2]-24

    outputs[:,-3] = 40*outputs[:,-3]-20
    outputs[:,-2] = 48*outputs[:,-2]-24

    print(test_feats)

    for idx, feat in enumerate(test_feats):
        print("Real features : ", feat)
        print("Predicted features : ", outputs[idx].detach().numpy())

        
        in_coord = outputs[idx,:2]
        in_m = outputs[idx,2]
        targ_coord = feat[:2]
        targ_m = feat[2]

        mloss = float(10*F.mse_loss(torch.mul(targ_m,in_coord.T).T, targ_coord))
        bloss = float(F.binary_cross_entropy(in_m, targ_m))
        print("Loss : ", mloss, bloss, mloss+bloss)
        tot_loss+= mloss+bloss
    test_loss.append(tot_loss/n_test_samples)


tensor([[-2.5299e+00, -2.5861e+00,  1.0000e+00],
        [ 5.7679e+00, -6.2227e-01,  1.0000e+00],
        [ 6.0071e+00,  7.3302e+00,  1.0000e+00],
        ...,
        [-5.9209e+00,  1.6237e+01,  1.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.6419e-03,  5.0107e+00,  1.0000e+00]])
Real features :  tensor([-2.5299, -2.5861,  1.0000])
Predicted features :  [-5.8202   -4.994301  1.      ]
Loss :  83.1264877319336 0.0 83.1264877319336
Real features :  tensor([ 5.7679, -0.6223,  1.0000])
Predicted features :  [ 6.905426  -1.4522934  1.       ]
Loss :  9.914342880249023 0.0 9.914342880249023
Real features :  tensor([6.0071, 7.3302, 1.0000])
Predicted features :  [3.132059 8.910015 1.      ]
Loss :  53.80842208862305 0.0 53.80842208862305
Real features :  tensor([-6.6664, -4.8819,  1.0000])
Predicted features :  [-7.6287384 -7.6570396  0.9999999]
Loss :  43.13832092285156 1.1920930376163597e-07 43.138321042060866
Real features :  tensor([ 5.7311, -0.8997,  1.0000])
P

In [ ]:
print(test_loss)